Lib import

In [ ]:
import numpy as np
import pandas as pd
import json
from PIL import Image
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

Runtime settings

In [ ]:
#    Data-Splitting
TRAIN_DATASET_PERCENTAGE_SIZE = 70
VALIDATION_DATASET_PERCENTAGE_SIZE = 15
#    Image conversion
IMAGE_SIZE = 224
#    Hyperparams
EPOCHS = 7          # 1 E = 3 datasets (see below)
BATCH_SIZE = 32
LEARNING_RATE = 0.0001
BETA_1 = 0.9
BETA_2 = 0.999
EPSILON = 10e-8
L2_REGULARIZATION_PENALTY = 0
#   RANDOM
RANDOM_SEED = 2021

random.seed(RANDOM_SEED)

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
DEVICE

Directories

In [ ]:
#--- INPUTS ---
#model
inpmodel_path = "../input/pretrained-pytorch-models/resnet50-19c8e357.pth"

#Root
root_dir = "../input/cassava-leaf-disease-classification"

#Train
train_dataset_dir = os.path.join(root_dir, "train_images")
train_csv_path = os.path.join(root_dir, "train.csv")

#Test
test_dataset_dir = os.path.join(root_dir, "test_images")

#--- OUTPUTS ---
#Model
out_dir = "./"
model_path = os.path.join(out_dir, "cassava_saved_model.pth")
model_fixed = os.path.join(out_dir, "cassava_saved_model_fixed.pth")

#Generated Meta
TRAINDATASET_train_path = os.path.join(out_dir, "trainds_train.csv")
TRAINDATASET_validation_path = os.path.join(out_dir, "trainds_validation.csv")
TRAINDATASET_test_path = os.path.join(out_dir, "trainds_test.csv")

wrong_preds_csv_path = os.path.join(out_dir, "wrong_predictions.csv")

fixed_train_csv_path = os.path.join(out_dir, "fixed_train.csv")

TRAINDATASET_fixed_train_path = os.path.join(out_dir, "trainds_fixed_train.csv")
TRAINDATASET_fixed_validation_path = os.path.join(out_dir, "trainds_fixed_validation.csv")
TRAINDATASET_fixed_test_path = os.path.join(out_dir, "trainds_fixed_test.csv")

sub_filepath = os.path.join(out_dir, "submission.csv")

#predicted test
testfiles_predictions_csv_path = os.path.join(out_dir, "testfiles_predictions.csv")

In [ ]:
#Get full train list
train_full = pd.read_csv(train_csv_path)
train_full

Analize data

In [ ]:
#Calc class objects
class_0_items = 0
class_1_items = 0
class_2_items = 0
class_3_items = 0
class_4_items = 0
for index, row in train_full.iterrows():
    if row['label'] == 0:
      class_0_items += 1
    elif row['label'] == 1:
      class_1_items += 1
    elif row['label'] == 2:
      class_2_items += 1
    elif row['label'] == 3:
      class_3_items += 1
    elif row['label'] == 4:
      class_4_items += 1
#Calc %
summary_items = len(train_full.index)
class_0_perc = (100 * class_0_items)//summary_items
class_1_perc = (100 * class_1_items)//summary_items
class_2_perc = (100 * class_2_items)//summary_items
class_3_perc = (100 * class_3_items)//summary_items
class_4_perc = (100 * class_4_items)//summary_items

In [ ]:
#show class % diagram
values = np.array([class_0_perc, class_1_perc, class_2_perc, class_3_perc, class_4_perc])
value_labels = ["Class 0 - " + str(class_0_perc) + "%", "Class 1 - " + str(class_1_perc) + "%", "Class 2 - " + str(class_2_perc) + "%", "Class 3 - " + str(class_3_perc) + "%", "Class 4 - " + str(class_4_perc) + "%"]
plt.pie(values, labels = value_labels)
plt.legend(title = "Legend:")
plt.show() 

Separating test set

In [ ]:
if (TRAIN_DATASET_PERCENTAGE_SIZE + VALIDATION_DATASET_PERCENTAGE_SIZE) >= 100:
  raise SPLITSIZEERROR("Choose correct dataset sizes!")

#shuffle full dataset
train_full = shuffle(train_full)

#count items
summary_items = len(train_full.index)
train_items = (summary_items * TRAIN_DATASET_PERCENTAGE_SIZE)//100
validation_items = (summary_items * VALIDATION_DATASET_PERCENTAGE_SIZE)//100
test_items = summary_items - (train_items + validation_items)

#make 3 sets
train_dataset_csv = train_full.iloc[ 0 : (train_items-1) ]
validation_dataset_csv = train_full.iloc[ (train_items-1) : (train_items-1) + (validation_items-1) ]
test_dataset_csv = train_full.iloc[ (train_items-1) + (validation_items-1) : summary_items + 1]

In [ ]:
train_dataset_csv.to_csv(TRAINDATASET_train_path)
train_dataset_csv

In [ ]:
validation_dataset_csv.to_csv(TRAINDATASET_validation_path)
validation_dataset_csv

In [ ]:
test_dataset_csv.to_csv(TRAINDATASET_test_path)
test_dataset_csv

Making dataset vectors

In [ ]:
X_Train = train_dataset_csv['image_id'].values
Y_Train = train_dataset_csv['label'].values

X_Val = validation_dataset_csv['image_id'].values
Y_Val = validation_dataset_csv['label'].values

X_Test = test_dataset_csv['image_id'].values
Y_Test = test_dataset_csv['label'].values

Pick Transformations

In [ ]:
#Pick random picture
randomrow_index = random.randint(0, summary_items)
randomrow = train_full.iloc[randomrow_index]
randomname = randomrow['image_id']
randomimg = Image.open(os.path.join(train_dataset_dir, randomname))

#raw picked image
plt.axis('off')
plt.imshow(randomimg)

In [ ]:
#Resize IMAGE_SIZE
trasformation_Res = transforms.Resize((IMAGE_SIZE, IMAGE_SIZE))
pic_tr1 = trasformation_Res(randomimg)

plt.axis('off')
plt.imshow(pic_tr1)

In [ ]:
#RandomRotation
ANGLE = 180
trasformation_Rot = transforms.RandomRotation(ANGLE)
pic_tr2 = trasformation_Rot(randomimg)

plt.axis('off')
plt.imshow(pic_tr2)

In [ ]:
#RandomHorizontalFlip
PROBABILITY = 0.5
trasformation_HFlip = transforms.RandomHorizontalFlip(p=PROBABILITY)
pic_tr3 = trasformation_HFlip(randomimg)

plt.axis('off')
plt.imshow(pic_tr3)

In [ ]:
#Resize in HALF
Size_Width, Size_Height = randomimg.size
HalfSize_Width = Size_Width//2
HalfSize_Height = Size_Height//2
transform_ResHalf = transforms.Resize((HalfSize_Height, HalfSize_Width))

pic_reshalf = transform_ResHalf(randomimg)

plt.axis('off')
plt.imshow(pic_reshalf)

In [ ]:
#Random Crop
transform_RCrop = transforms.RandomCrop(size=(224, 224))
pic_rcrop = transform_RCrop(randomimg)

plt.axis('off')
plt.imshow(pic_rcrop)

Defining transformation sets:

In [ ]:
#Raw
transformations_0 = transforms.Compose(
    [transforms.ToTensor(),
     trasformation_Res,
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
transformations_1 = transforms.Compose(
    [transforms.ToTensor(),
     trasformation_Res,
     trasformation_Rot,
     trasformation_HFlip,
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
#transformations_1 DEMO
pic_tr_all = trasformation_HFlip(trasformation_Rot(trasformation_Res(randomimg)))

plt.axis('off')
plt.imshow(pic_tr_all)

In [ ]:
transformations_2 = transforms.Compose(
    [transforms.ToTensor(),
     transform_ResHalf,
     transform_RCrop,
     trasformation_Rot,
     trasformation_HFlip,
     transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])

In [ ]:
#transformations_2 DEMO
pic_tr_all = trasformation_HFlip(trasformation_Rot(transform_RCrop(transform_ResHalf(randomimg))))

plt.axis('off')
plt.imshow(pic_tr_all)

Using custom dataset class

In [ ]:
class GetData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        x = Image.open(os.path.join(self.dir, self.fnames[index]))
        if "train" in self.dir:            
            return self.transform(x), self.lbs[index]            
        elif "test" in self.dir:            
            return self.transform(x), self.fnames[index]

Making Torch readable datasets

In [ ]:
#Resize only
trainset_0 = GetData(train_dataset_dir, X_Train, Y_Train, transformations_0)
train_0_loader = DataLoader(trainset_0, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Transformations set 1
trainset_1 = GetData(train_dataset_dir, X_Train, Y_Train, transformations_1)
train_1_loader = DataLoader(trainset_1, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Transformations set 2
trainset_2 = GetData(train_dataset_dir, X_Train, Y_Train, transformations_2)
train_2_loader = DataLoader(trainset_2, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Valid
validset = GetData(train_dataset_dir, X_Val, Y_Val, transformations_0)
validloader = DataLoader(validset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

#Test
testset = GetData(train_dataset_dir, X_Test, Y_Test, transformations_0)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

Model init

In [ ]:
model = torchvision.models.resnet50()
model.load_state_dict(torch.load(inpmodel_path, map_location=torch.device(DEVICE)))
model.fc = nn.Linear(2048, 5, bias=True)
model = model.to(DEVICE)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2), eps=EPSILON, weight_decay=L2_REGULARIZATION_PENALTY)

Training

In [ ]:
epochs = EPOCHS
min_valid_loss = np.inf
for e in range(epochs):
    # TRAINING
    train_loss = 0.0
    model.train()
    # Set 0
    for data, labels in train_0_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #Set 1
    for data, labels in train_1_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #Set 2
    for data, labels in train_2_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #
    # VALIDATION
    valid_loss = 0.0
    model.eval()
    for data, labels in validloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        target = model(data)
        loss = loss_func(target,labels)
        valid_loss = loss.item() * data.size(0)
    #
    # PRINT INFO
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_0_loader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), model_path)

Testing on validation

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in validloader:
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy (validation): %d" %(100 * correct/total))

Testing on test

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in testloader:
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy (test): %d" %(100 * correct/total))

# Post analysis

Use all items to form dataset to test wrong predictions

In [ ]:
X_full_train = train_full['image_id'].values
Y_full_train = train_full['label'].values

In [ ]:
fulltrainset = GetData(train_dataset_dir, X_full_train, Y_full_train, transformations_0)
fulltrainsetloader = DataLoader(fulltrainset, batch_size=1, shuffle=False, num_workers=4)

Test predictions

In [ ]:
wrong_predictions = pd.DataFrame(columns=['image_id', 'predicted', 'neuron_value', 'ground_truth', 'neuron_value_on_gt', 'neuron_value_diff'])
wrong_predictions

In [ ]:
with torch.no_grad():
    model.eval()
    i = 0
    for data in fulltrainsetloader:
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        outputs = model(inputs)
        current_image = train_full['image_id'].iloc[i]
        current_image_class = train_full['label'].iloc[i]
        neuron_value, predicted = torch.max(outputs.data, 1)
        neuron_value_on_gt = outputs[0][current_image_class].item()
        neuron_value_diff = neuron_value.item() - neuron_value_on_gt
        if not predicted == current_image_class:
            #write to csv: current_image, predicted, neuron_value
            wrong_predictions = wrong_predictions.append({'image_id':current_image, 'predicted':predicted.item(),'neuron_value':neuron_value.item(),'ground_truth':current_image_class,'neuron_value_on_gt':neuron_value_on_gt,'neuron_value_diff':neuron_value_diff}, ignore_index=True)
        i += 1

In [ ]:
wrong_predictions.to_csv(wrong_preds_csv_path)
summary_items_wrong = len(wrong_predictions.index)
wrong_predictions

See which class network mistakenly makes choice

In [ ]:
#Calc class objects
check1_class_0_items = 0
check1_class_1_items = 0
check1_class_2_items = 0
check1_class_3_items = 0
check1_class_4_items = 0
for index, row in wrong_predictions.iterrows():
    if row['predicted'] == 0:
      check1_class_0_items += 1
    elif row['predicted'] == 1:
      check1_class_1_items += 1
    elif row['predicted'] == 2:
      check1_class_2_items += 1
    elif row['predicted'] == 3:
      check1_class_3_items += 1
    elif row['predicted'] == 4:
      check1_class_4_items += 1
#Calc %
check1_class_0_perc = (100 * check1_class_0_items)//summary_items_wrong
check1_class_1_perc = (100 * check1_class_1_items)//summary_items_wrong
check1_class_2_perc = (100 * check1_class_2_items)//summary_items_wrong
check1_class_3_perc = (100 * check1_class_3_items)//summary_items_wrong
check1_class_4_perc = (100 * check1_class_4_items)//summary_items_wrong

In [ ]:
# Show class % diagram
check1_values = np.array([check1_class_0_perc, check1_class_1_perc, check1_class_2_perc, check1_class_3_perc, check1_class_4_perc])
check1_value_labels = ["Class 0 - " + str(check1_class_0_perc) + "%", "Class 1 - " + str(check1_class_1_perc) + "%", "Class 2 - " + str(check1_class_2_perc) + "%", "Class 3 - " + str(check1_class_3_perc) + "%", "Class 4 - " + str(check1_class_4_perc) + "%"]
plt.pie(check1_values, labels = check1_value_labels)
plt.legend(title = "Legend:")
plt.show()

See in which class network is mistaken the most

In [ ]:
#Calc class objects
check2_class_0_items = 0
check2_class_1_items = 0
check2_class_2_items = 0
check2_class_3_items = 0
check2_class_4_items = 0
for index, row in wrong_predictions.iterrows():
    if row['predicted'] == 0:
      check2_class_0_items += 1
    elif row['predicted'] == 1:
      check2_class_1_items += 1
    elif row['predicted'] == 2:
      check2_class_2_items += 1
    elif row['predicted'] == 3:
      check2_class_3_items += 1
    elif row['predicted'] == 4:
      check2_class_4_items += 1
#Calc %
check2_class_0_perc = (100 * check2_class_0_items)//summary_items_wrong
check2_class_1_perc = (100 * check2_class_1_items)//summary_items_wrong
check2_class_2_perc = (100 * check2_class_2_items)//summary_items_wrong
check2_class_3_perc = (100 * check2_class_3_items)//summary_items_wrong
check2_class_4_perc = (100 * check2_class_4_items)//summary_items_wrong

In [ ]:
# Show class % diagram
check2_values = np.array([check2_class_0_perc, check2_class_1_perc, check2_class_2_perc, check2_class_3_perc, check2_class_4_perc])
check2_value_labels = ["Class 0 - " + str(check2_class_0_perc) + "%", "Class 1 - " + str(check2_class_1_perc) + "%", "Class 2 - " + str(check2_class_2_perc) + "%", "Class 3 - " + str(check2_class_3_perc) + "%", "Class 4 - " + str(check2_class_4_perc) + "%"]
plt.pie(check2_values, labels = check2_value_labels)
plt.legend(title = "Legend:")
plt.show()

Get visual info

In [ ]:
values_diff = wrong_predictions['neuron_value_diff'].values

In [ ]:
diff_min = values_diff.min()
print('Min value: ' + str(diff_min))
diff_max = values_diff.max()
print('Max value: ' + str(diff_max))
diff_avg = np.average(values_diff)
print('Average value: ' + str(diff_avg))

In [ ]:
plt.hist(values_diff, bins=20)

Execluding certain values where network is super sure that dataset is wrong

In [ ]:
exclude_train = pd.DataFrame(columns=['image_id', 'label'])
exclude_train

In [ ]:
for index, row in wrong_predictions.iterrows():
    if row['neuron_value_diff'] >= 2: # 2 was chosen as it will drop about 5-6% of presumably wrong dataset
        image_id = row['image_id']
        label = row['ground_truth']
        exclude_train = exclude_train.append({'image_id':image_id, 'label':label}, ignore_index=True)

In [ ]:
exclude_train

In [ ]:
fixed_train = pd.DataFrame(columns=['image_id', 'label'])
fixed_train

In [ ]:
for index_exc, row_exc in exclude_train.iterrows():
    index = train_full[train_full['image_id'] == row_exc['image_id']].index
    train_full.drop(index, inplace=True)
fixed_train = train_full

In [ ]:
fixed_train.to_csv(fixed_train_csv_path)
summary_items_fixed = len(fixed_train.index)
fixed_train

# Retraining model with new training dataset

Analize data

In [ ]:
#Calc class objects
retrain_class_0_items = 0
retrain_class_1_items = 0
retrain_class_2_items = 0
retrain_class_3_items = 0
retrain_class_4_items = 0
for index, row in fixed_train.iterrows():
    if row['label'] == 0:
      retrain_class_0_items += 1
    elif row['label'] == 1:
      retrain_class_1_items += 1
    elif row['label'] == 2:
      retrain_class_2_items += 1
    elif row['label'] == 3:
      retrain_class_3_items += 1
    elif row['label'] == 4:
      retrain_class_4_items += 1
#Calc %
retrain_class_0_perc = (100 * retrain_class_0_items)//summary_items_fixed
retrain_class_1_perc = (100 * retrain_class_1_items)//summary_items_fixed
retrain_class_2_perc = (100 * retrain_class_2_items)//summary_items_fixed
retrain_class_3_perc = (100 * retrain_class_3_items)//summary_items_fixed
retrain_class_4_perc = (100 * retrain_class_4_items)//summary_items_fixed

In [ ]:
# Show class % diagram
retrain_values = np.array([retrain_class_0_perc, retrain_class_1_perc, retrain_class_2_perc, retrain_class_3_perc, retrain_class_4_perc])
retrain_value_labels = ["Class 0 - " + str(retrain_class_0_perc) + "%", "Class 1 - " + str(retrain_class_1_perc) + "%", "Class 2 - " + str(retrain_class_2_perc) + "%", "Class 3 - " + str(retrain_class_3_perc) + "%", "Class 4 - " + str(retrain_class_4_perc) + "%"]
plt.pie(retrain_values, labels = retrain_value_labels)
plt.legend(title = "Legend:")
plt.show()

Separating into sets

In [ ]:
#shuffle full dataset
fixed_train = shuffle(fixed_train)

#count items
retrain_train_items = (summary_items_fixed * TRAIN_DATASET_PERCENTAGE_SIZE)//100
retrain_validation_items = (summary_items_fixed * VALIDATION_DATASET_PERCENTAGE_SIZE)//100
retrain_test_items = summary_items_fixed - (retrain_train_items + retrain_validation_items)

#make 3 sets
retrain_train_dataset_csv = fixed_train.iloc[ 0 : (retrain_train_items-1) ]
retrain_validation_dataset_csv = fixed_train.iloc[ (retrain_train_items-1) : (retrain_train_items-1) + (retrain_validation_items-1) ]
retrain_test_dataset_csv = fixed_train.iloc[ (retrain_train_items-1) + (retrain_validation_items-1) : summary_items_fixed + 1]

In [ ]:
retrain_train_dataset_csv.to_csv(TRAINDATASET_fixed_train_path)
retrain_train_dataset_csv

In [ ]:
retrain_validation_dataset_csv.to_csv(TRAINDATASET_fixed_validation_path)
retrain_validation_dataset_csv

In [ ]:
retrain_test_dataset_csv.to_csv(TRAINDATASET_fixed_test_path)
retrain_test_dataset_csv

Making dataset vectors

In [ ]:
X_Train_retrain = retrain_train_dataset_csv['image_id'].values
Y_Train_retrain = retrain_train_dataset_csv['label'].values

X_Val_retrain = retrain_validation_dataset_csv['image_id'].values
Y_Val_retrain = retrain_validation_dataset_csv['label'].values

X_Test_retrain = retrain_test_dataset_csv['image_id'].values
Y_Test_retrain = retrain_test_dataset_csv['label'].values

Making Torch readable datasets

In [ ]:
#Resize only
retrain_trainset_0 = GetData(train_dataset_dir, X_Train_retrain, Y_Train_retrain, transformations_0)
retrain_train_0_loader = DataLoader(retrain_trainset_0, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Transformations set 1
retrain_trainset_1 = GetData(train_dataset_dir, X_Train_retrain, Y_Train_retrain, transformations_1)
retrain_train_1_loader = DataLoader(retrain_trainset_1, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Transformations set 2
retrain_trainset_2 = GetData(train_dataset_dir, X_Train_retrain, Y_Train_retrain, transformations_2)
retrain_train_2_loader = DataLoader(retrain_trainset_2, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

#Valid
retrain_validset = GetData(train_dataset_dir, X_Val_retrain, Y_Val_retrain, transformations_0)
retrain_validloader = DataLoader(retrain_validset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

#Test
retrain_testset = GetData(train_dataset_dir, X_Test_retrain, Y_Test_retrain, transformations_0)
retrain_testloader = DataLoader(retrain_testset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

Model init

In [ ]:
model = torchvision.models.resnet50()
model.load_state_dict(torch.load(inpmodel_path, map_location=torch.device(DEVICE)))
model.fc = nn.Linear(2048, 5, bias=True)
model = model.to(DEVICE)

In [ ]:
loss_func = nn.CrossEntropyLoss()
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2), eps=EPSILON, weight_decay=L2_REGULARIZATION_PENALTY)

Retraining

In [ ]:
epochs = EPOCHS
min_valid_loss = np.inf
for e in range(epochs):
    # TRAINING
    train_loss = 0.0
    model.train()
    # Set 0
    for data, labels in retrain_train_0_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #Set 1
    for data, labels in retrain_train_1_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #Set 2
    for data, labels in retrain_train_2_loader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        optimiser.zero_grad()
        target = model(data)
        loss = loss_func(target,labels)
        loss.backward()
        optimiser.step()
        train_loss = loss.item() * data.size(0)
    #
    # VALIDATION
    valid_loss = 0.0
    model.eval()
    for data, labels in retrain_validloader:
        if torch.cuda.is_available():
            data, labels = data.cuda(), labels.cuda()
        
        target = model(data)
        loss = loss_func(target,labels)
        valid_loss = loss.item() * data.size(0)
    #
    # PRINT INFO
    print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_0_loader)} \t\t Validation Loss: {valid_loss / len(validloader)}')
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased({min_valid_loss:.6f}--->{valid_loss:.6f}) \t Saving The Model')
        min_valid_loss = valid_loss
        # Saving State Dict
        torch.save(model.state_dict(), model_fixed)

Testing on test

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for data in retrain_testloader:
        inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("Accuracy retrain (test): %d" %(100 * correct/total))

# TestSet predict

In [ ]:
X_Test_pred = [name for name in (os.listdir(test_dataset_dir))]

In [ ]:
testset_pred = GetData(test_dataset_dir, X_Test_pred, None, transformations_0)
testloader_pred = DataLoader(testset_pred, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

In [ ]:
to_submit = []
correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for image, fname in testloader_pred: 
        image = image.to(DEVICE)
        
        logits = model(image)        
        ps = torch.exp(logits)        
        _, top_class = ps.topk(1, dim=1)
        
        for pred in top_class:
            to_submit.append([fname[0], pred.item()])

In [ ]:
sub = pd.DataFrame.from_records(to_submit, columns=['image_id', 'label'])
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)